### 基于 MessagePassing 构建图神经网络

In [1]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

### 定义图神经网络模型

GCNConv 的数学定义为 :

$$
\mathbf{x}_{i}^{(k)}=\sum_{j \in \mathcal{N}(i) \cup\{i\}} \frac{1}{\sqrt{\operatorname{deg}(i)} \cdot \sqrt{\operatorname{deg}(j)}} \cdot\left(\boldsymbol{\Theta} \cdot \mathbf{x}_{j}^{(k-1)}\right)
$$

----

其中，邻接节点的表征$x_{j}^{(k-1)}$首先通过与权重矩阵 相乘进行变换，然后按端点的度 $deg(i), deg(j)$
进行归一化处理，最后进行求和。这个公式可以分为以下几个步骤 :

1. 向邻接矩阵添加自环边。
2. 对节点表征做线性转换。
3. 计算归一化系数。
4. 归一化邻接节点的节点表征。
5. 将相邻节点表征相加（"求和 "聚合）

步骤1-3通常是在消息传递发生之前计算的。
步骤4-5可以使用MessagePassing基类轻松处理。该层的全部实现如下所示。

In [2]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr="add", flow="source_to_target")
        # aggr : 聚合方式 => add
        # flow="source_to_target" : 表示消息从源节点传播到目标节点
        self.linear = torch.nn.Linear(in_features=in_channels, out_features=out_channels)

    def forward(self, x, edge_index):
        """
        :param x: [num_nodes, in_channels]
        :param edge_index: [2, num_edges]
        :return:
        """
        # step 1 : 向邻接矩阵添加自环边
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # step 2 : 对节点表征做线性变换
        x = self.linear(x)
        # step 3 : 计算归一化系数
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # step 4-5 : 开启消息传播
        return self.propagate(edge_index=edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        """
        :param x_j: [num_edges, out_channels]
        :param norm: 归一化系数
        :return:
        """
        return norm.view(-1, 1) * x_j

模型详解 :

1. GCNConv继承了MessagePassing并以"求和"作为领域节点信息聚合方式。
2. 该层的所有 逻 辑 都 发 生 在 其 forward() 方法中
3. 在这 里, 我 们 首 先 使 用 torch_geometric.utils.add_self_loops()函数向我们的边索引添加自循环边（步骤1）
4. 以及通过调用torch.nn.Linear实例对节点表征进行线性变换（步骤2）。
5. propagate()方法也在forward方法中被调用，propagate()方法被调用后节点间的信息传递开始执行
6. 归一化系数是由每个节点的节点度得出的，它被转换为每条边的节点度。结果被保存在形状为[num_edges,]的变量norm中（步骤3）
7. 在message()方法中，我们需要通过norm对邻接节点表征x_j进行归一化处理

通过以上内容的学习，我们便掌握了创建一个仅包含一次“消息传递过程”的图神经
网络的方法。如下方代码所示，我们可以很方便地初始化和调用它：

In [3]:
from torch_geometric.datasets import Planetoid

datasets = Planetoid(root='../../datasets/Cora', name="Cora")
# 获取第一张图
data = datasets[0]
print("num_features : {}, ")
# 创建模型 : GCNConv(节点特征, out_channel)
net = GCNConv(data.num_features, 64)
# x : 节点属性矩阵，大小为 [num_nodes, num_node_features]
# edge_index : 边索引矩阵，大小为[2, num_edges]
# 第 0 行可称为头（head）节点、源（source）节点、邻接节点，第 1 行可称为尾（tail）节点、目标（target）节点、中心节点
h_nodes = net(data.x, data.edge_index)
print(h_nodes.shape)

torch.Size([2708, 64])


通过串联多个这样的简单图神经网络，我们就可以构造复杂的图神经网络模型

### 重写message方法

前面我们介绍了，传递给propagate()方法的参数，如果是节点的属性的话，可以 被拆分成属于中心节点的部分和属于邻接节点的部分, 只需在变量名后面加上_i或 _j。
现在我们有一个额外的节点属性，节点的度deg, 我们希望message方法还能接 收中心节点的度，我们对前面GCNConv的message方法进行改造得到新的GCNConv类

In [9]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr="add", flow="source_to_target")
        # aggr : 聚合方式 => add
        # flow="source_to_target" : 表示消息从源节点传播到目标节点
        self.linear = torch.nn.Linear(in_features=in_channels, out_features=out_channels)

    def forward(self, x, edge_index):
        """
        :param x: [num_nodes, in_channels]
        :param edge_index: [2, num_edges]
        :return:
        """
        # step 1 : 向邻接矩阵添加自环边
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # step 2 : 对节点表征做线性变换
        x = self.linear(x)
        # step 3 : 计算归一化系数
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # step 4-5 : 开启消息传播
        return self.propagate(edge_index=edge_index, x=x, norm=norm, deg=deg.view(-1, 1))

    def message(self, x_j, norm, deg_i):
        """

        :param x_j: [num_edges, out_channels]
        :param norm: 归一化系数
        :param deg_i:
        :return:
        """
        return norm.view(-1, 1) * x_j * deg_i

In [11]:
from torch_geometric.datasets import Planetoid

datasets = Planetoid(root='../../datasets/Cora', name="Cora")
# 获取第一张图
data = datasets[0]
print("num_features : {}, ".format(data.num_features))
# 创建模型 : GCNConv(节点特征, out_channel)
net = GCNConv(data.num_features, 64)
# x : 节点属性矩阵，大小为 [num_nodes, num_node_features]
# edge_index : 边索引矩阵，大小为[2, num_edges]
# 第 0 行可称为头（head）节点、源（source）节点、邻接节点，第 1 行可称为尾（tail）节点、目标（target）节点、中心节点
h_nodes = net(data.x, data.edge_index)
print(h_nodes.shape)

num_features : 1433, 
torch.Size([2708, 64])


若一个数据可以被拆分成属于中心节点的部分和属于邻接节点的部分，其形状必须 是[num_nodes, *]，因此在上方代码, 我们执行了deg.view(-1, 1)操作，使得数据形状为[num_nodes, 1]，然后才将数据传给propagate()方法。

### aggregate方法的覆写

在前面的例子的基础上，我们增加如下的aggregate方法。通过观察运行结果我们 可 以 看 到 ， 我 们 覆 写 的 aggregate 方 法 被 调 用 ， 同 时 在 super(GCNConv, self).__init__(aggr='add')中传递给aggr参数的值被存储到了self.aggr属性中。

In [13]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr="add", flow="source_to_target")
        # aggr : 聚合方式 => add
        # flow="source_to_target" : 表示消息从源节点传播到目标节点
        self.linear = torch.nn.Linear(in_features=in_channels, out_features=out_channels)

    def forward(self, x, edge_index):
        """
        :param x: [num_nodes, in_channels]
        :param edge_index: [2, num_edges]
        :return:
        """
        # step 1 : 向邻接矩阵添加自环边
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # step 2 : 对节点表征做线性变换
        x = self.linear(x)
        # step 3 : 计算归一化系数
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # step 4-5 : 开启消息传播
        return self.propagate(edge_index=edge_index, x=x, norm=norm, deg=deg.view(-1, 1))

    def message(self, x_j, norm, deg_i):
        """

        :param x_j: [num_edges, out_channels]
        :param norm: 归一化系数
        :param deg_i:
        :return:
        """
        return norm.view(-1, 1) * x_j * deg_i
    def aggregate(self, inputs, index, ptr, dim_size):
        """
        :param inputs:
        :param index:
        :param ptr:
        :param dim_size:
        :return:
        """
        print('self.aggr:', self.aggr)
        print("`aggregate` is called")
        return super().aggregate(inputs, index, ptr=ptr, dim_size=dim_size)


from torch_geometric.datasets import Planetoid

datasets = Planetoid(root='../../datasets/Cora', name="Cora")
# 获取第一张图
data = datasets[0]
print("num_features : {}, ".format(data.num_features))
# 创建模型 : GCNConv(节点特征, out_channel)
net = GCNConv(data.num_features, 64)
# x : 节点属性矩阵，大小为 [num_nodes, num_node_features]
# edge_index : 边索引矩阵，大小为[2, num_edges]
# 第 0 行可称为头（head）节点、源（source）节点、邻接节点，第 1 行可称为尾（tail）节点、目标（target）节点、中心节点
h_nodes = net(data.x, data.edge_index)
print(h_nodes.shape)

num_features : 1433, 
self.aggr: add
`aggregate` is called
torch.Size([2708, 64])


### message_and_aggregate方法的覆写

在一些案例中，“消息传递”与“消息聚合”可以融合在一起。对于这种情况，我们可以覆写message_and_aggregate方法，在message_and_aggregate方法中一块实现“消息传递”与“消息聚合”，这样能使程序的运行更加高效。